In [ ]:
%%capture
!pip install -U -q transformers[torch]
!pip install -U -q bitsandbytes trl peft

In [ ]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

In [ ]:
from transformers  import AutoModelForCausalLM
from transformers import AutoTokenizer


max_seq_length = 2048
dtype = None


model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b",
    load_in_4bit = False,
    token = hf_token
)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task paired with an input that provides the context. Write a response that correctly completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
inst = "Classify the following user input into one of positive, negative or neutral."
input = "Zaggle Prepaid Standalone December 2023 Net Sales at Rs 199.51 crore, up 35.12% Y-o-Y"


inputs = tokenizer(
[
    alpaca_prompt.format(
        inst, # instruction
        input, # input
        "",
    )
], return_tensors = "pt").to("cuda")
model = model.to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 10)
result = tokenizer.batch_decode(outputs)[0]
print (result)

<bos>Below is an instruction that describes a task paired with an input that provides the context. Write a response that correctly completes the request.

### Instruction:
Classify the following user input into one of positive, negative or neutral.

### Input:
Zaggle Prepaid Standalone December 2023 Net Sales at Rs 199.51 crore, up 35.12% Y-o-Y

### Response:
The user input is positive.<eos>


In [ ]:
from transformers import AutoTokenizer
from collections import OrderedDict

def collect_tokens(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    vocab = tokenizer.get_vocab()

    # Sort the vocabulary by token ID
    sorted_vocab = sorted(vocab.items(), key=lambda x: x[1])

    # Tokens that match 'positive', 'negative', or 'neutral' when lowercased
    sentiment_tokens = []
    for token, token_id in vocab.items():
        if token.lower() in ['positive', 'negative']:
            sentiment_tokens.append((token, token_id))

    # Combine tokens and remove duplicates while preserving order
    unique_tokens = list(OrderedDict.fromkeys(sentiment_tokens))

    # Count total tokens
    total_tokens = len(unique_tokens)

    return total_tokens, unique_tokens

total_count, tokens_and_ids = collect_tokens("Telugu-LLM-Labs/Telugu-gemma-7b-finetuned-sft")

print(f"Total number of tokens: {total_count}")

print("Tokens and their token IDs:")
for token, token_id in tokens_and_ids:
    print(f"Token: {token}, Token ID: {token_id}")

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

Total number of tokens: 8
Tokens and their token IDs:
Token: neutral, Token ID: 56347
Token: Neutral, Token ID: 70874
Token: negative, Token ID: 31827
Token: POSITIVE, Token ID: 166788
Token: Positive, Token ID: 35202
Token: positive, Token ID: 30212
Token: Negative, Token ID: 39654
Token: NEGATIVE, Token ID: 197553


In [ ]:
model

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaR

In [ ]:
import torch

# Create a mapping between the original token IDs and the new token IDs
token_id_mapping = {token_id: new_token_id for new_token_id, (token, token_id) in enumerate(tokens_and_ids)}

print (token_id_mapping)

{56347: 0, 70874: 1, 31827: 2, 166788: 3, 35202: 4, 30212: 5, 39654: 6, 197553: 7}


In [ ]:
# Get the last embedding decoder layer
last_embedding_decoder_layer = model.lm_head

# Create a new last embedding decoder layer with the subset of tokens_and_ids
new_last_embedding_decoder_layer = torch.nn.Linear(last_embedding_decoder_layer.in_features, len(tokens_and_ids))

# Copy the weights of the subset of tokens_and_ids from the original last embedding decoder layer
for token, token_id in tokens_and_ids:
    new_token_id = token_id_mapping[token_id]
    new_last_embedding_decoder_layer.weight.data[new_token_id] = last_embedding_decoder_layer.weight.data[token_id]

# Set the new last embedding decoder layer in the model
model.lm_head = new_last_embedding_decoder_layer

In [ ]:
model

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaR

In [ ]:
# Create a reverse mapping from new token IDs to their corresponding tokens
reverse_token_id_mapping = {new_token_id: token for new_token_id, (token, _) in enumerate(tokens_and_ids)}

print (reverse_token_id_mapping)

{0: 'neutral', 1: 'Neutral', 2: 'negative', 3: 'POSITIVE', 4: 'Positive', 5: 'positive', 6: 'Negative', 7: 'NEGATIVE'}


In [ ]:
%%time
# inst = "Classify the following user input into one of positive, negative or neutral."
inst = "Classify the following user input (good, bad or normal)"

input = "Zaggle Prepaid Standalone December 2023 Net Sales at Rs 199.51 crore, up 35.12% Y-o-Y"
# input="India Tourism D Standalone December 2023 Net Sales at Rs 133.51 crore, up 14.31% Y-o-Y"
# input="Paytm Payments Bank customers can no longer deposit money into their accounts.Users will no longer get salary credits, direct benefit transfers, and subsidies in their Paytm Payments Bank account."

inputs = tokenizer(
[
    alpaca_prompt.format(
        inst, # instruction
        input, # input
        "",
    )
], return_tensors = "pt").to("cuda")

model = model.to("cuda")

input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Generate the output tokens
outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=1,
    return_dict_in_generate=True,
    output_scores=True,
)

# Get the generated token IDs (excluding the input tokens)
generated_token_ids = outputs.sequences[:, input_ids.shape[-1]:]

print(generated_token_ids)

# Decode the generated token IDs to their corresponding tokens
decoded_output = [reverse_token_id_mapping[token_id.item()] for token_id in generated_token_ids[0]]


print(' '.join(decoded_output))

tensor([[4]], device='cuda:0')
Positive
CPU times: user 104 ms, sys: 552 µs, total: 105 ms
Wall time: 230 ms
